<a href="https://colab.research.google.com/github/Shravani703/Shravani703/blob/main/Item_Based_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd

In [2]:
books=pd.read_csv('/content/drive/MyDrive/Ops Final/Books.csv')
ratings=pd.read_csv('/content/drive/MyDrive/Ops Final/Ratings.csv')
users=pd.read_csv('/content/drive/MyDrive/Ops Final/Users.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
books.head()

In [ ]:
users.head()

In [ ]:
ratings.head()

In [ ]:
print(ratings.shape)
print(books.shape)
print(users.shape)

(1149780, 3)
(271360, 8)
(278858, 3)


In [3]:
ratings_books = ratings.merge(books,on='ISBN')

In [ ]:
len(pd.unique(ratings_books['User-ID']))

In [ ]:
rbu_df = ratings_books.merge(users,on='User-ID')

In [ ]:
ratings_books.head()

**Cleaning the title - removes trailing spaces, converts to title case**

In [4]:
def clean_booktitle(title):
    return str(title).title().strip()
    
ratings_books['Book-Title'] = ratings_books['Book-Title'].apply(clean_booktitle)

**User constraint & book constraint - looping through each user & book**



In [ ]:
print("Shape before filtering:",ratings_books.shape)
ratings_books = ratings_books[ratings_books['User-ID'].map(ratings_books['User-ID'].value_counts()) > 200]

ratings_books = ratings_books[ratings_books['Book-Title'].map(ratings_books['Book-Title'].value_counts()) > 50]

ratings_books = ratings_books.reset_index(drop=True)
print("Shape after filtering:",ratings_books.shape)

Shape before filtering: (1031136, 10)
Shape after filtering: (58685, 10)


**New data frame with column users, row books. Each row is a vector.**

In [ ]:
data_matrix = ratings_books.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')
data_matrix.fillna(0,inplace=True)

In [ ]:
data_matrix.head()

In [ ]:
import random
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from PIL import Image
import requests
pd.set_option('max_colwidth', 1000)

In [ ]:
similarity_scores_books = cosine_similarity(data_matrix) 

**An array that contains n number of lists (if there are n number of books), list 1 has the cosine similarity between row1/book1 and every other row/book**

In [ ]:
similarity_scores_books


*   **book_name is the input variable. if we do not have the book title the user has entered return -1**
*  **index is an array of the position number of a book_name entered if the book exists in our database. The book '1984' for example is in position 1, therefore the index will return 0**
*  **enumerate function creates key value pairs of (counter, similarity score value), first book it evaluates will have a counter of 0 and so forth**
*  **lambda function sorts the list in descending order**
*  **[1:8] results second to ninth most similar book, first most similar book is the same book itself and is thereby ignored**

In [ ]:
def recommend_books(ratings_books,book_name):
    
    if ratings_books['Book-Title'].str.contains(book_name).any()==False:
        return -1
        
    index = np.where(data_matrix.index==book_name)[0][0]

    similar_items = list(enumerate(similarity_scores_books[index]))
    similar_items = sorted(similar_items,key = lambda x:x[1],reverse=True)[1:8]

    book_recommend = []
    for i in similar_items:
        temp_df = ratings_books[ratings_books['Book-Title'] == data_matrix.index[i[0]]]
        book_recommend.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))       
    return book_recommend
        

In [ ]:
recommend_books(ratings_books, "A Bend In The Road")

NameError: ignored

**KNN ALGORITHM**

In [ ]:
data_matrix.values

array([[ 9.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 10.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [ ]:
data_matrix.values.shape

(693, 810)

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
features_matrix = csr_matrix(data_matrix)

In [ ]:
model_NN = NearestNeighbors(n_neighbors=7, algorithm="brute", metric="cosine")

In [ ]:
model_NN.fit(X=features_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=7)

In [ ]:
distances, indices = model_NN.kneighbors(X=data_matrix.iloc[4, :].values.reshape(1, -1), n_neighbors=7, return_distance=True)

In [ ]:
distances

array([[3.33066907e-16, 7.04373882e-01, 7.08578584e-01, 7.39566496e-01,
        7.53683127e-01, 7.60635804e-01, 7.63477036e-01]])

In [ ]:
indices

array([[  4,  24, 556, 413, 179, 118, 353]])

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(data_matrix.index[4]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, data_matrix.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for A Bend In The Road:

1: A Walk To Remember, with distance of 0.7043738816822027:
2: The Last Time They Met : A Novel, with distance of 0.7085785835016092:
3: Sea Glass: A Novel, with distance of 0.7395664959548052:
4: Family Album, with distance of 0.7536831267601498:
5: Cradle And All, with distance of 0.7606358035877223:
6: Nights In Rodanthe, with distance of 0.7634770361435563:
